# Working With COG

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/main?filepath=docs%2Fexamples%2F%2Fnotebooks%2FWorking_with_CloudOptimizedGeoTIFF_simple.ipynb)

For this demo we will use the new `DigitalGlobe OpenData` dataset https://www.digitalglobe.com/ecosystem/open-data


#### Requirements
- folium
- httpx

`pip install folium httpx`

In [1]:
# Uncomment this line if you need to install the dependencies
# !pip install folium httpx

In [1]:
import json

import httpx

from folium import Map, TileLayer

%matplotlib inline

In [2]:
titiler_endpoint = "http://titiler:80"  # Developmentseed Demo endpoint. Please be kind.
url = "https://burn-severity.s3.us-east-2.amazonaws.com/public/geology/metrics.tif"

## Get COG Info

In [16]:
# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

bounds = r["bounds"]
r

{'bounds': [-115.68613103347651,
  35.22090516567677,
  -115.49088700466815,
  35.38820015150643],
 'minzoom': 11,
 'maxzoom': 13,
 'band_metadata': [['b1', {}], ['b2', {}], ['b3', {}], ['b4', {}], ['b5', {}]],
 'band_descriptions': [['b1', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'],
  ['b2', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'],
  ['b3', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'],
  ['b4', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'],
  ['b5', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf']],
 'dtype': 'float64',
 'nodata_type': 'None',
 'colorinterp': ['gray', 'undefined', 'undefined', 'undefined', 'undefined'],
 'driver': 'GTiff',
 'count': 5,
 'width': 978,
 'height': 838,
 'overviews': []}

## Get COG Metadata

In [4]:

# Increase the timeout value to 60 seconds
timeout = httpx.Timeout(60.0)

# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = httpx.get(
    f"{titiler_endpoint}/cog/statistics",
    params={
        "url": url,
    },
    timeout=timeout

).json()

print(json.dumps(r, indent=4))

{
    "b1": {
        "min": -0.0891973445986723,
        "max": 0.18173923830790226,
        "mean": 0.01304073944038668,
        "count": 452628.0,
        "sum": 5902.6038114233415,
        "std": 0.022217696023949785,
        "median": 0.01288976708168071,
        "majority": 0.0,
        "minority": -0.0891973445986723,
        "unique": 272484.0,
        "histogram": [
            [
                344.0,
                6009.0,
                71202.0,
                197574.0,
                148299.0,
                27181.0,
                1577.0,
                267.0,
                105.0,
                70.0
            ],
            [
                -0.0891973445986723,
                -0.06210368630801485,
                -0.03501002801735739,
                -0.007916369726699943,
                0.01917728856395752,
                0.04627094685461498,
                0.07336460514527242,
                0.10045826343592988,
                0.12755192172658736,
  

### Display Tiles

In [18]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Burn Severity",
).add_to(m)

m

In [26]:
import math
def deg2num(lat_deg, lon_deg, zoom):
  lat_rad = math.radians(lat_deg)
  n = 2.0 ** zoom
  xtile = int((lon_deg + 180.0) / 360.0 * n)
  ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
  return (xtile, ytile)

zoom = 11
lat_deg = 35.32
lon_deg = -115.6
xtile, ytile = deg2num(lat_deg, lon_deg, zoom)

print(f"XTile: {xtile}, YTile: {ytile}")

XTile: 366, YTile: 808


In [25]:
import requests 
import matplotlib.pyplot as plt
import io
from PIL import Image

z, x, y= 11, 0, 0  # Change with your tile indexes
url = f"http://titiler:80/cog/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?url=https%3A%2F%2Fburn-severity.s3.us-east-2.amazonaws.com%2Fpublic%2Fgeology%2Fmetrics.tif"

response = requests.get(url)

response

<Response [404]>

In [21]:
((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2)

(35.304552658591604, -115.58850901907233)

# Work with non-byte data

In [15]:

url = "https://burn-severity.s3.us-east-2.amazonaws.com/public/geology/metrics.tif"

# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

print(r)
print("Data is of type:", r["dtype"])

# This dataset has statistics metadata
r
# minv, maxv = r["band_metadata"][0][1]["STATISTICS_MINIMUM"], r["band_metadata"][0][1]["STATISTICS_MAXIMUM"]
# print("With values from ", minv, "to ", maxv)



{'bounds': [-115.68613103347651, 35.22090516567677, -115.49088700466815, 35.38820015150643], 'minzoom': 11, 'maxzoom': 13, 'band_metadata': [['b1', {}], ['b2', {}], ['b3', {}], ['b4', {}], ['b5', {}]], 'band_descriptions': [['b1', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'], ['b2', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'], ['b3', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'], ['b4', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'], ['b5', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf']], 'dtype': 'float64', 'nodata_type': 'None', 'colorinterp': ['gray', 'undefined', 'undefined', 'undefined', 'undefined'], 'driver': 'GTiff', 'count': 5, 'width': 978, 'height': 838, 'overviews': []}
Data is of type: float64


{'bounds': [-115.68613103347651,
  35.22090516567677,
  -115.49088700466815,
  35.38820015150643],
 'minzoom': 11,
 'maxzoom': 13,
 'band_metadata': [['b1', {}], ['b2', {}], ['b3', {}], ['b4', {}], ['b5', {}]],
 'band_descriptions': [['b1', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'],
  ['b2', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'],
  ['b3', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'],
  ['b4', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf'],
  ['b5', 'stackstac-a8aa11363bdc88c7456ad28d429dc8cf']],
 'dtype': 'float64',
 'nodata_type': 'None',
 'colorinterp': ['gray', 'undefined', 'undefined', 'undefined', 'undefined'],
 'driver': 'GTiff',
 'count': 5,
 'width': 978,
 'height': 838,
 'overviews': []}

In [34]:
# We could get the min/max values using the statistics endpoint
r = httpx.get(
    f"{titiler_endpoint}/cog/statistics",
    params = {
        "url": url,
    }
).json()

print(json.dumps(r["b1"], indent=4))

ReadTimeout: timed out

### Display Tiles


Note: By default if the metadata has `min/max` statistics, titiler will use those to rescale the data

In [12]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"] + 1
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Swisstopo"
).add_to(m)
m

Apply ColorMap

Now that the data is rescaled to byte values (0 -> 255) we can apply a colormap

In [13]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "rescale": f"{minv},{maxv}",
        "colormap_name": "terrain"
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"] + 1
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Swisstopo"
).add_to(m)
m

Apply non-linear colormap (intervals)

see https://cogeotiff.github.io/rio-tiler/colormap/#intervals-colormaps

In [14]:
import json

cmap = json.dumps(
    [
        # ([min, max], [r, g, b, a])
        ([0, 1500], [255,255,204, 255]),
        ([1500, 1700], [161,218,180, 255]),
        ([1700, 1900], [65,182,196, 255]),
        ([1900, 2000], [44,127,184, 255]),
        ([2000, 3000], [37,52,148, 255]),
    ]
)
# https://colorbrewer2.org/#type=sequential&scheme=YlGnBu&n=5

r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "colormap": cmap
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"] + 1
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Swisstopo"
)
aod_layer.add_to(m)
m